# Pytorch Basics

In [2]:
import torch
import torchvision
import torch.nn as nn
import numpy as np
import torchvision.transforms as transforms

### Basic autograd example 1

In [3]:
# create tensors
x = torch.tensor(1., requires_grad=True)
w = torch.tensor(2., requires_grad=True)
b = torch.tensor(3., requires_grad=True)

# build computational graph
y = w * x + b

# compute gradients
y.backward()

# print out the gradients
print(x.grad)
print(w.grad)
print(b.grad)

tensor(2.)
tensor(1.)
tensor(1.)


### Basic autograd example 2

In [4]:
# create tensors of shape (10, 3) and (10, 2)
x = torch.randn(10, 3)
y = torch.randn(10, 2)

# bulid a fully connected layer
linear = nn.Linear(3, 2)
print("w:", linear.weight)
print("b:", linear.bias)

w: Parameter containing:
tensor([[-0.5001,  0.4295, -0.2755],
        [ 0.2553,  0.4466, -0.1246]], requires_grad=True)
b: Parameter containing:
tensor([-0.1649,  0.4973], requires_grad=True)


In [5]:
# build loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(linear.parameters(), lr=0.01)

# forward pass
pred = linear(x)

# compute loss
loss = criterion(pred, y)
print("loss ", loss.item())

loss  0.6573954224586487


In [6]:
# backward pass
loss.backward()

# print out the gradients
print("dL/dw: ", linear.weight.grad)
print("dL/db: ", linear.bias.grad)

dL/dw:  tensor([[-0.0106, -0.0426, -0.1510],
        [ 0.0352,  0.6837, -0.2607]])
dL/db:  tensor([-0.0388,  0.3035])


In [7]:
# 1-step gradient descent
optimizer.step()

# can also perform gradient descent at the low level
# linear.weight.data.sub_(0.01 * linear.weight.grad.data)
# linear.bias.data.sub_(0.01 * linear.bias.grad.data)

# print out the loss after 1 step gradient descent
pred = linear(x)
loss = criterion(pred, y)
print('loss after 1 step optimization: ', loss.item())

loss after 1 step optimization:  0.6508762836456299


### Loading data from numpy

In [8]:
# create a numpy array
x = np.array([[1, 2], [3, 4]])

# convert the numpy array to a torch tensor
y = torch.from_numpy(x)

# convert the torch tensor to a numpy array
z = y.numpy()

### Input pipeline

In [9]:
# download and construct CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='../../data/', train=True,
                                             transform=transforms.ToTensor(),
                                             download=True)

# Fetch one data pair (read data from disk)
image, label = train_dataset[0]
print(image.size())
print(label)


Extracting ../../data/cifar-10-python.tar.gz to ../../data/
torch.Size([3, 32, 32])
6


In [10]:
# data loader (provides queues and threads in a very simple way)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=64,
                                           shuffle=True)

# when iteration starts, queue and thread start to load data from files
data_iter = iter(train_loader)

# Mini-batch images and labels
images, labels = data_iter.next()

# actual usage of the data loader is as below
for images, labels in train_loader:
    # training code here
    pass

### Input pipeline for custom dataset

In [12]:
# building custom dataset 
class CustomDataset(torch.utils.data.Dataset):
  def __init__(self):
      # TODO
      # 1. initialize file paths or a list of file names
      pass
  def __getitem__(self, index):
      # TODO
      # 1. read one data from file (e.g. using numpy.fromfile, PIL.Image.open)
      # 2. preprocess the data (e.g. torchvision.Transform)
      # 3. return a data pair (e.g. image and label)
      pass
  def __len__(self):
      # return the total size of the dataset
      return 999

# then use the prebuild data loader
custom_dataset = CustomDataset()
train_loader = torch.utils.data.DataLoader(dataset=custom_dataset,
                                           batch_size=64,
                                           shuffle=True)



### Pretrained model

In [13]:
# download and load the pretrained ResNet-18
resnet = torchvision.models.resnet18(pretrained=True)

# to fine tune only the top later, set as below
for param in resnet.parameters():
    param.requires_grad = False
  
# replace the top layer for fine tuning
resnet.fc = nn.Linear(resnet.fc.in_features, 100) # 100 is examplary

# Forward pass
images = torch.rand(64, 3, 224 , 224)
outputs = resnet(images)
print(outputs.size())

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth



torch.Size([64, 100])


### Save and Load the model

In [ ]:
# save and load the entire model
torch.save(resnet, "model.ckpt")
model = torch.load("model.ckpt")

# save and load only the model parameters (recommended)
torch.save(resnet.state_dict(), 'params.ckpt')
resnet.load_state_dict(torch.load('params.ckpt'))